In [1]:
import sys
import os
import os.path as osp

sys.path.insert(0, "../utils/")
sys.path.insert(0, "../")

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

In [3]:
from utils.read_config import yaml_to_object
config = yaml_to_object("./../config.yaml")

root_dir = osp.abspath(osp.dirname("."))

setattr(config, "root_dir", osp.dirname(root_dir))

data_path = osp.join(config.root_dir, config.data.MPII.path.base)

training_annotation_file = osp.join(data_path, config.data.MPII.path.annotations.training)
validation_annotation_file = osp.join(data_path, config.data.MPII.path.annotations.validation)
image_dir = osp.join(data_path, config.data.MPII.path.images)

In [4]:
from torchvision import transforms
from torchvision.io import read_image

In [5]:
from utils.data import MPIIAnnotationHandler

In [6]:
dh = MPIIAnnotationHandler(training_annotation_file, validation_annotation_file, image_dir)

Loading annotation data...
Done! (t=3.60s)


In [7]:
image_filename, keypoints, visible, center, scale, _ = dh.get_annotation(0)

H = W = 200 * scale

In [8]:
img = read_image(image_filename).float()/255.0

In [10]:
H_des = W_des = 256

In [12]:
resizer = transforms.Resize((H_des, W_des))
img = resizer(img)


In [13]:
img.shape

torch.Size([3, 256, 256])

In [ ]:
t = transforms.Compose([
    transforms.Lambda(lambda image_tensor: x.float()/255.0),
    transforms.Resize((H_des, W_des)),
    transforms.ColorJitter(brightness=0.3, contrast=0.5, saturation=0.5, hue=0.2),
    transforms.RandomHorizontalFlip(p=0.5),
])


def data_transformer(img):
    img = img.float()/255.0
    img = transforms.functional.resize(img, size=(256, 256))
    img = transforms.functional.adjust_hue(img, hue_factor=.2)  # heu_factor = [-0.2, 0.2] 
    img = transforms.functional.adjust_brightness(img, brightness_factor=1.0)  #[max(0, 1 - brightness), 1 + brightness]
    img = transforms.functional.adjust_contrast(img, contrast_factor=0.2)
    img = transforms.functional.hflip(img)
    